In [2]:
# Remved warnings 
import warnings  
warnings.filterwarnings('ignore')

# This packages are used for file handling
import os
import os.path
import random 
import pickle 
from tqdm import tqdm
from datetime import datetime

# Used for plotting and visualizations
%matplotlib inline 
import seaborn as sns 
import matplotlib.pyplot as plt 

# For linear algebra and arrays manipulation 
import cv2 
import math
import random 
import pandas as pd
import numpy as np
from numpy import save, load, asarray


# Building machine learning model 
import keras
import tensorflow as tf 
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.compat.v1 import ConfigProto, InteractiveSession
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import losses
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import mobilenet_v2, mobilenet, densenet
from tensorflow.keras.layers import (Dense, MaxPooling2D, Conv2D, Flatten, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D,
    DepthwiseConv2D, Dropout, ReLU, Concatenate, DepthwiseConv2D, Input, add,  Conv2DTranspose)


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, LearningRateScheduler
# Config variables
from config import * 

In [3]:
# Initialize CUDA
config = ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [8]:
# Initialize WANDB dashboard 
import wandb
from wandb.keras import WandbCallback

os.environ["WANDB_NOTEBOOK_NAME"] = "main.ipynb"
os.environ["WANDB_API_KEY"] = '2257bf21347391d8f93c8f9f7d620cbbfbf9dcc5'
wandb.login()
wandb.init(project="Stochastic Gradient Descent AlexNet", entity="coder-ph")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [9]:

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='constant'
)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    'data/training',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical'
)
validation_dataget = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_directory(
    'data/testing',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical'
)

Found 30401 images belonging to 9 classes.
Found 14857 images belonging to 9 classes.


In [15]:
EPOCHS=100
LEARNING_RATE = 0.001
DECAY_RATE = LEARNING_RATE/EPOCHS
MOMENTUM =  0.8	


# Initalize Callbacks 
#ModelCheckpoint callback saves a model at some interval. 
filepath="saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.ckpt" #File name includes epoch and validation accuracy.

	#Use Mode = max for accuracy and min for loss. 


checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=11, verbose=1)
#This callback will stop the training when there is no improvement in
# the validation loss for three consecutive epochs.

#CSVLogger logs epoch, acc, loss, val_acc, val_loss
log_csv = CSVLogger('csv-files/nihogalog_logs_alexnet-sigmoid.csv', separator=',', append=False)

#######################################################################
#CUSTOM LEARNING RATE
#Time based decay: lr *= (1. / (1. + decay * iterations))

#Step decay drops the learning rate by a factor every few epochs.: 
# lr = lr0 * drop^floor(epoch / epochs_drop) 
#import math
#def step_decay(epoch):
#   initial_lrate = 0.1
#   drop = 0.5
#   epochs_drop = 10.0
#   lrate = initial_lrate * math.pow(drop,  
#           math.floor((1+epoch)/epochs_drop))
#   return lrate


#Eexponential decay
#To be used instead of static learning rate of 0.1
# 𝑙𝑟=𝑙𝑟₀ × 𝑒**(−𝑘𝑡)
#lr can still be used as initial value. 


#NOTE: Redefine model with a new name to make sure we start from scratch

def exp_decay(epoch):
    lrate = LEARNING_RATE  * np.exp(-DECAY_RATE*EPOCHS)
    return lrate
lr_rate = LearningRateScheduler(exp_decay,verbose=0)



callbacks = [
    checkpoint, 
    early_stop, 
    log_csv,
    lr_rate,
    WandbCallback(dataset_name="Resnet")
]

In [16]:


ACTIVATION = 'sigmoid'
def AlexNet(shape, classes):
    model = keras.models.Sequential([
        # Layer 0 
        Conv2D(filters=96, kernel_size=(11,11), strides=(2,2), activation=ACTIVATION, input_shape=shape),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),

        # Layer 1 
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=ACTIVATION, padding="same"),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),
        
        # Layer 2 
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=ACTIVATION, padding="same"),
        BatchNormalization(),
        
        # Layer 3 
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=ACTIVATION, padding="same"),
        BatchNormalization(),

        # Layer 4 
        Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation=ACTIVATION, padding="same"),
        BatchNormalization(),
        MaxPool2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        
        # Layer 5 
        Dense(4096, activation=ACTIVATION),
        Dropout(0.5),
        
        # Layer 6 
        Dense(4096, activation=ACTIVATION),
        Dropout(0.5),
        
        Dense(classes, activation='softmax')
    ])
    return model 
    
model = AlexNet(shape=INPUT_SHAPE, classes=len(CATEGORIES))
sgd = tf.keras.optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay=DECAY_RATE, nesterov=False)
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=sgd,
    metrics=['accuracy'],
)

history = model.fit( train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/100
949/951 [============================>.] - ETA: 0s - loss: 2.0674 - accuracy: 0.2308
Epoch 1: val_accuracy improved from -inf to 0.43703, saving model to saved_models\weights-improvement-01-0.44.ckpt
INFO:tensorflow:Assets written to: saved_models\weights-improvement-01-0.44.ckpt\assets
951/951 [==============================] - 42s 43ms/step - loss: 2.0667 - accuracy: 0.2309 - val_loss: 1.4600 - val_accuracy: 0.4370 - lr: 9.9900e-04 - _timestamp: 1649042804.0000 - _runtime: 3127.0000
Epoch 2/100
950/951 [============================>.] - ETA: 0s - loss: 1.4431 - accuracy: 0.4309
Epoch 2: val_accuracy improved from 0.43703 to 0.45433, saving model to saved_models\weights-improvement-02-0.45.ckpt
INFO:tensorflow:Assets written to: saved_models\weights-improvement-02-0.45.ckpt\assets
951/951 [==============================] - 41s 43ms/step - loss: 1.4431 - accuracy: 0.4310 - val_loss: 1.4501 - val_accuracy: 0.4543 - lr: 9.9900e-04 - _timestamp: 1649042846.0000 - _runtime: 316

In [23]:
model.save('save-models/alexnet-sigmoid-final')

INFO:tensorflow:Assets written to: save-models/alexnet-sigmoid-final\assets


In [21]:
x = pd.DataFrame(history.history)
x.head()

,loss,accuracy,val_loss,val_accuracy,lr,_timestamp,_runtime
0,2.066725,0.230946,1.459983,0.437033,0.000999,1649042804,3127
1,1.443119,0.430973,1.450114,0.454331,0.000999,1649042846,3169
2,1.194997,0.503898,1.045459,0.565929,0.000999,1649042890,3213
3,1.040411,0.572679,1.055142,0.587804,0.000999,1649042932,3255
4,0.879505,0.660274,0.944346,0.685064,0.000999,1649042974,3297


In [22]:
x.to_csv('alexnet-sigmoid-final.csv')